In [ ]:
#Importing libraries
import os
from langchain_community.document_loaders import TextLoader, WebBaseLoader, PDFMinerLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.vectorstores import Chroma
from groq import Groq
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
import requests
import re
import gradio as gr


In [ ]:
#Setting the environment
os.environ["GROQ_API_KEY"] = "GROQ_API_KEY"

In [ ]:
#Loading the constitution
url = "https://www.parliament.go.ke/sites/default/files/2017-05/The_Constitution_of_Kenya_2010.pdf"
r = requests.get(url)
with open("constitution.pdf", "wb") as f:
    f.write(r.content)
    
loader = PDFMinerLoader("constitution.pdf")
docs_list = loader.load() 

In [ ]:
#Splitting the document into chuncks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500,
    chunk_overlap=500,
    separators=["ARTICLE", "CHAPTER", "\n\n"])

doc_splits = text_splitter.split_documents(docs_list)

In [5]:
#Embedding the documents and storing  them in a vector store
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=doc_splits,          
    embedding=embeddings,
    persist_directory="constitution_index"
)

retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 3
    }
)

In [6]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature = 0,
    max_tokens=500
)

In [7]:
prompt = PromptTemplate(
    template="""
You are an assistant for question-answering Kenyan constitution tasks.
Use the following documents to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Documents: {documents}
Answer:
""",
input_variables=["question", "documents"]
)



In [8]:
rag_chain = prompt | llm | StrOutputParser()

In [9]:
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain

    def run(self, question):
        documents = self.retriever.invoke(question)
        doc_texts = "\n".join([doc.page_content for doc in documents])
        
        # Get the answer from the LLM
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer


In [10]:
rag_app = RAGApplication(retriever, rag_chain)

question = "What are grounds for losing a seat?"
answer = rag_app.run(question)

print("Answer:", answer)

Answer: According to Article 103 of the Constitution of Kenya, 2010, the grounds for losing a seat in Parliament are:

(a) if the member dies;
(b) if, during any session of Parliament, the member is absent from eight sittings of the relevant House without permission, in writing, from the Speaker, and is unable to offer a satisfactory explanation for the absence to the relevant committee;
(c) if the member is otherwise removed from office under this Constitution or legislation enacted under Article 80;
(d) if the member resigns from Parliament in writing to the Speaker;
(e) if, having been elected to Parliament—
(i) as a member of a political party, the member resigns from that party or is deemed to have resigned from the party as determined in accordance with the legislation contemplated in clause (2); or
(ii) as an independent candidate, the member joins a political party;
(f) at the end of the term of the relevant House; or
(g) if the member becomes disqualified for election to Parli

In [11]:
def ask(question):
    answer = rag_app.run(question)
    print("Answer:", answer)



In [12]:
ask("Which rights are guaranteed under the Bill of Rights, and how does the Constitution protect these rights?")

Answer: The Bill of Rights in the Kenyan Constitution guarantees the following rights:

1. Right to life (Article 26)
2. Equality and freedom from discrimination (Article 27)
3. Human dignity (Article 28)
4. Freedom and security of the person (Article 29)
5. Freedom from slavery, servitude, and forced labor (Article 30)
6. Right to privacy (Article 31)
7. Freedom of conscience, religion, belief, and opinion (Article 32)
8. Freedom of expression (Article 33)
9. Freedom of the media (Article 34)
10. Right to access information (Article 35)
11. Freedom of association (Article 36)
12. Right to assemble, demonstrate, picket, and petition (Article 37)
13. Political rights (Article 38)
14. Freedom of movement and residence (Article 39)
15. Protection of the right to property (Article 40)
16. Labor relations (Article 41)
17. Right to a clean and healthy environment (Article 42)
18. Economic and social rights (Article 43)
19. Right to language and culture (Article 44)
20. Right to family (Artic

In [13]:
ask("What are the types of lands recognized?")

Answer: According to the Constitution of Kenya, 2010, the types of lands recognized are:

1. **Public land**: This includes land that belongs to the people of Kenya collectively as a nation, communities, and individuals. Public land is classified into several categories, including:
	* Land that was unalienated government land at the effective date.
	* Land lawfully held, used, or occupied by any State organ.
	* Land transferred to the State by way of sale, reversion, or surrender.
	* Land in respect of which no individual or community ownership can be established.
	* Land in respect of which no heir can be identified.
	* Minerals and mineral oils.
	* Government forests, game reserves, water catchment areas, national parks, animal sanctuaries, and specially protected areas.
	* Roads and thoroughfares.
	* Rivers, lakes, and other water bodies.
	* The territorial sea, exclusive economic zone, and seabed.
	* The continental shelf.
	* Land between the high and low water marks.
	* Any land n

In [16]:
def gradio_interface(query: str):
    answer = rag_app.run(query)
    return (answer
    )

# Build the Gradio app
gui = gr.Interface(
    fn=gradio_interface,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Markdown(),
    title="Kenyan Constitution RAG",
    description="Provide a query and receive a response based on the Kenyan Constitution",
)

# Launch the app
if __name__ == "__main__":
    gui.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
